In [1]:
import pandas as pd
import numpy as np
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from collections import Counter

from itertools import chain

In [2]:
# nltk.download()

In [3]:
violations_section = pd.read_csv("../data/violations_section.csv")
violations_TCA = pd.read_csv("../data/violations_TCA.csv")
violations_MCL = pd.read_csv("../data/violations_MCL.csv")
violations_description_id = pd.read_csv("../data/violations_description_id.csv")
violations_remainder = pd.read_csv("../data/violations_remainder.csv")

In [4]:
violations_list = [violations_section, violations_TCA, violations_MCL, violations_description_id, violations_remainder]

In [5]:
for violation in violations_list:
    assert 'Unnamed: 0' in violation.columns

In [6]:
for violation in violations_list:
    violation.set_index('Unnamed: 0', inplace=True)
    violation.index.rename('Original_Index', inplace=True)

In [7]:
violations_description_id.head()

,id,type,desc,text,remedial
Original_Index,,,,,
15,(B). It is unlawful for any person to dispose ...,NaN,N,NaN,1.0
438,he or she shall serve notice of the risk and t...,NaN,NaN,NaN,NaN
439,tree at the expense of the property owner,NaN,N,NaN,2.0
707,"human occupation or use, upon failure or refus...",NaN,NaN,NaN,NaN
708,"remove or demolish, such dwelling or structure...",NaN,N,NaN,2.0


In [8]:
violations_description_id['text'] = violations_description_id['id'].values

In [9]:
violations_description_id.head()

,id,type,desc,text,remedial
Original_Index,,,,,
15,(B). It is unlawful for any person to dispose ...,NaN,N,(B). It is unlawful for any person to dispose ...,1.0
438,he or she shall serve notice of the risk and t...,NaN,NaN,he or she shall serve notice of the risk and t...,NaN
439,tree at the expense of the property owner,NaN,N,tree at the expense of the property owner,2.0
707,"human occupation or use, upon failure or refus...",NaN,NaN,"human occupation or use, upon failure or refus...",NaN
708,"remove or demolish, such dwelling or structure...",NaN,N,"remove or demolish, such dwelling or structure...",2.0


In [10]:
for violation in violations_list:
    violation.drop('id', axis='columns', inplace=True)
    
    violation['desc'].replace(np.nan, '', regex=True, inplace=True)
    violation['text'].replace(np.nan, '', regex=True, inplace=True)

In [11]:
violations_section.head()

,type,desc,text,remedial
Original_Index,,,,
0,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS
1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER
2,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT
3,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT
4,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE


In [12]:
violations_TCA.head()

,type,desc,text,remedial
Original_Index,,,,
106,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG
376,CAAR_LAND_REG,LANDLORD REGISTRATION REQUIRED,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,NaN
807,BT575101A2,Not Manufacturer and Wholesale Retail,T.C.A 57-5-101(a)(2): Except as otherwise prov...,NaN
808,BT575101A3,Not Wholesale and Manufacturer Retail,T.C.A 57-5-101(a)(3): Except as otherwise prov...,NaN
809,BT575103A1,Operating without county city permit,T.C.A 57-5-103(a)(1): It is unlawful to operat...,NaN


In [13]:
violations_MCL.head()

,type,desc,text,remedial
Original_Index,,,,
481,CAGA_DEX,DOG EXCREMENT,MCL Chapter 8.04.180 – Removal of Excrement. ...,NaN
482,CAGB_HAZ_W,HAZARDOUS WASTE,"MCL Chapter 10.20.150 - Hazardous, pathogenic ...",NaN
483,CAGC_GCL,GARBAGE CONTAINER LIDS,MCL Chapter 10.20.160 – Container Requirements...,NaN
484,CAGD_ID,ILLEGAL DUMPING,MCL Chapter 10.20.320 – Illegal Dumping. It i...,NaN
485,CAGE_LPR,LITTER ON PRIVATE PROPERTY,MCL Chapter 10.24.070 - Litter on private prop...,NaN


In [14]:
violations_description_id.head()

,type,desc,text,remedial
Original_Index,,,,
15,NaN,N,(B). It is unlawful for any person to dispose ...,1.0
438,NaN,,he or she shall serve notice of the risk and t...,NaN
439,NaN,N,tree at the expense of the property owner,2.0
707,NaN,,"human occupation or use, upon failure or refus...",NaN
708,NaN,N,"remove or demolish, such dwelling or structure...",2.0


In [15]:
violations_remainder.head()

,type,desc,text,remedial
Original_Index,,,,
126,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN


In [16]:
master_violations = pd.concat(violations_list)

In [17]:
master_violations

,type,desc,text,remedial
Original_Index,,,,
0,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS
1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER
2,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT
3,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT
4,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE
5,BOARDINGRQ,Boarding Requirements,Section 16.24.340 (U) (2) – Boarding of Vacant...,CAAH_BOARDING_REQU
6,BOARDVAC,Boarding of Vacant Buildings,Section 16.24.340 (U) (1) – Boarding of Vacant...,CAAH_BOARDING_VABLDG
7,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT
8,CERTCOMP,Certificate of Compliance Required,Section 17.40.580 - Certificate of Compliance:...,CAAZ_CERT_COMP


In [18]:
def tokenize_column(column_values):
    return word_tokenize(column_values)
    
def tokenize_row(index, row):
    desc = row['desc']
    text = row['text']
    
    desc_tokenized, text_tokenized = [], []
    
    if desc:
        desc_tokenized = tokenize_column(desc)
        
    if text:
        text_tokenized = tokenize_column(text)

    return (index, desc_tokenized, text_tokenized)

In [19]:
tokenized_data = list()

for index, row in master_violations.iterrows():
    tokenized_row_data = tokenize_row(index, row)
    tokenized_data.append(tokenized_row_data)

In [20]:
tokenized_df = pd.DataFrame(tokenized_data, columns=['Original_Index', 'tokenized_desc', 'tokenized_text'])
tokenized_df.set_index('Original_Index', inplace=True)

tokenized_df.head()

,tokenized_desc,tokenized_text
Original_Index,,
0,[Banners],"[Section, 17.32.060, &, 17.32.070, -, Banners,..."
1,"[Proper, Maintenance, Req, -, Owner]","[Section, 16.16.030, (, B, ), -, Proper, maint..."
2,"[Proper, Maintenance, Req]","[Section, 16.16.030, (, A, ), -, Proper, maint..."
3,"[Building, Permit, Required]","[Section, 16.28.010, -, Building, Permit, Requ..."
4,"[Scope, of, Building, Code]","[Section, 16.16.020, (, B, ), -, The, provisio..."


In [21]:
master_violations = master_violations.join(tokenized_df)
master_violations.head()

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
0,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,[Banners],"[Section, 17.32.060, &, 17.32.070, -, Banners,..."
1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER,"[Proper, Maintenance, Req, -, Owner]","[Section, 16.16.030, (, B, ), -, Proper, maint..."
2,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT,"[Proper, Maintenance, Req]","[Section, 16.16.030, (, A, ), -, Proper, maint..."
3,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT,"[Building, Permit, Required]","[Section, 16.28.010, -, Building, Permit, Requ..."
4,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE,"[Scope, of, Building, Code]","[Section, 16.16.020, (, B, ), -, The, provisio..."


In [22]:
#subset that contains rows with the regular expression "Section #." in text column
master_section = master_violations[master_violations.text.str.match(r"(S|s)ection\s(\d+)\.", na=False)]

#new df where rows containing "Section #" are removed
master_nosection = master_violations.drop(master_section.index)

#subset that contains rows with variations of TCA in text column
master_TCA = master_nosection[master_nosection.text.str.match("(T.C.A|Tennessee Code Annotated)", na=False)]

#new df where rows containing "T.C.A" or Tennessee Code Annotated" are removed
master_noTCA = master_nosection.drop(master_TCA.index)

#subset that contains variations of MCL in text column 
#Be aware MCL and MCL Chapter appear to be different types of codes
master_MCL = master_noTCA[master_noTCA.text.str.match("(M.C.L|MCL Chapter)", na=False)]

#new df where rows containing "M.C.L" or "MCL Chapter" are removed
master_remainder = master_noTCA.drop(master_MCL.index)
#master_remainder

In [23]:
#export csv files containing subsets for section, TCA, MCL, descriptions in id column, and remaining entries.
master_section.to_csv("../data/master_section.csv")
master_TCA.to_csv("../data/master_TCA.csv")
master_MCL.to_csv("../data/master_MCL.csv")
master_remainder.to_csv("../data/master_remainder.csv")

In [81]:
master_MCL

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
481,CAGA_DEX,DOG EXCREMENT,MCL Chapter 8.04.180 – Removal of Excrement. ...,NaN,"[DOG, EXCREMENT]","[MCL, Chapter, 8.04.180, –, Removal, of, Excre..."
482,CAGB_HAZ_W,HAZARDOUS WASTE,"MCL Chapter 10.20.150 - Hazardous, pathogenic ...",NaN,"[HAZARDOUS, WASTE]","[MCL, Chapter, 10.20.150, -, Hazardous, ,, pat..."
483,CAGC_GCL,GARBAGE CONTAINER LIDS,MCL Chapter 10.20.160 – Container Requirements...,NaN,"[GARBAGE, CONTAINER, LIDS]","[MCL, Chapter, 10.20.160, –, Container, Requir..."
484,CAGD_ID,ILLEGAL DUMPING,MCL Chapter 10.20.320 – Illegal Dumping. It i...,NaN,"[ILLEGAL, DUMPING]","[MCL, Chapter, 10.20.320, –, Illegal, Dumping,..."
485,CAGE_LPR,LITTER ON PRIVATE PROPERTY,MCL Chapter 10.24.070 - Litter on private prop...,NaN,"[LITTER, ON, PRIVATE, PROPERTY]","[MCL, Chapter, 10.24.070, -, Litter, on, priva..."
486,CAGF_LPU,LITTER ON PUBLIC PROPERTY,MCL Chapter 10.24.140 - Litter on vacant lots....,NaN,"[LITTER, ON, PUBLIC, PROPERTY]","[MCL, Chapter, 10.24.140, -, Litter, on, vacan..."
487,CAGG_LVH,LITTER FROM VEHICLES,MCL Chapter 10.24.150 - Litter from vehicles--...,NaN,"[LITTER, FROM, VEHICLES]","[MCL, Chapter, 10.24.150, -, Litter, from, veh..."
488,CAGH_HWG,EXCESS VEGETATION,MCL Chapter 10.26.010 – Every premise includin...,NaN,"[EXCESS, VEGETATION]","[MCL, Chapter, 10.26.010, –, Every, premise, i..."
489,CAGI_GCAN,GARBAGE APPROVED CONTAINERS,MCL Chapter 10.32.120 - All garbage and trash ...,NaN,"[GARBAGE, APPROVED, CONTAINERS]","[MCL, Chapter, 10.32.120, -, All, garbage, and..."


In [83]:
MCL_MCL = master_MCL[master_MCL.text.str.match(r"M.C.L")]
MCL_MCL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 735 to 881
Data columns (total 6 columns):
type              76 non-null object
desc              76 non-null object
text              76 non-null object
remedial          0 non-null object
tokenized_desc    76 non-null object
tokenized_text    76 non-null object
dtypes: object(6)
memory usage: 4.2+ KB


In [26]:
MCL_Chapter = master_MCL[master_MCL.text.str.match(r"MCL Chapter")]

In [27]:
MCL_list = MCL_MCL["tokenized_desc"].tolist()
MCL_flat_list = [item for sublist in MCL_list for item in sublist]
#MCL_set = set(MCL_flat_list)
#MCL_set

MCL_kw_freq = dict(Counter(MCL_flat_list))
MCL_kw_freq

MCL_kw_s = pd.Series(MCL_kw_freq, name="freq")

MCL_kw = pd.DataFrame(data = MCL_kw_s, columns = ["freq"])
MCL_kw = MCL_kw.reset_index()
MCL_kw.columns = ["kw", "freq"]
#MCL_kw.head()
MCL_kw_sorted = MCL_kw.sort_values(by = "freq", ascending = False)
MCL_kw_sorted.head()

#results = []
#for tokenlist in MCL_list:
#    sentence_results = []
#    for token in tokenlist:
#        sentence_results.append(word_tokenize(token))
#    results.append(sentence_results)

#Counter(chain(MCL_list))

,kw,freq
12,BB,51
127,permit,9
119,on,8
0,",",6
153,sales,6


In [62]:
searchfor = [
    'BB ', 'sale', 'caterer', 'beer', 'moral', 'application', 
    "dist", "patron", "attire", "privilege", "suspension", "reproductions"]
MCL_drop = MCL_MCL.drop(MCL_MCL[MCL_MCL["desc"].str.contains('|'.join(searchfor), case = False)].index)

#s[s.str.contains('|'.join(searchfor))]

#MCL_noBB = MCL_MCL.drop(MCL_MCL[MCL_MCL["desc"].str.contains("BB ")].index)


In [50]:
MCL_MCL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 735 to 881
Data columns (total 6 columns):
type              76 non-null object
desc              76 non-null object
text              76 non-null object
remedial          0 non-null object
tokenized_desc    76 non-null object
tokenized_text    76 non-null object
dtypes: object(6)
memory usage: 6.7+ KB


In [48]:
Chapter_list = MCL_Chapter["tokenized_desc"].tolist()
Chapter_flat_list = [item for sublist in Chapter_list for item in sublist]


Chapter_kw_freq = dict(Counter(Chapter_flat_list))
#Chapter_kw_freq

Chapter_kw_s = pd.Series(Chapter_kw_freq, name="freq")

Chapter_kw = pd.DataFrame(data = Chapter_kw_s, columns = ["freq"])
Chapter_kw = Chapter_kw.reset_index()
Chapter_kw.columns = ["kw", "freq"]
#MCL_kw.head()
Chapter_kw_sorted = Chapter_kw.sort_values(by = "freq", ascending = False)
Chapter_kw_sorted.head()

,kw,freq
19,GARBAGE,5
39,SEWAGE,4
36,RAT,4
21,HARBORAGE,4
0,",",3


In [61]:
searchfor_2 = [
    "garbage", "sew", "vermin", "water", "plumbing", "excrement", 
    "litter", "junk", "waste", "dumping", "vegetation", "chickens"]
Chapter_drop = MCL_Chapter.drop(
    MCL_Chapter[MCL_Chapter["desc"].str.contains('|'.join(searchfor_2), case = False)].index)
MCL_Chapter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 481 to 724
Data columns (total 6 columns):
type              25 non-null object
desc              25 non-null object
text              25 non-null object
remedial          0 non-null object
tokenized_desc    25 non-null object
tokenized_text    25 non-null object
dtypes: object(6)
memory usage: 2.0+ KB


In [60]:
print(master_remainder)

                                type  \
Original_Index                         
15                               NaN   
438                              NaN   
439                              NaN   
707                              NaN   
708                              NaN   
895                              NaN   
897                              NaN   
898                              NaN   
899                              NaN   
900                              NaN   
901                              NaN   
902                              NaN   
905                              NaN   
908                              NaN   
913                              NaN   
914                              NaN   
917                              NaN   
919                              NaN   
920                              NaN   
924                              NaN   
931                              NaN   
126                         MVENVEND   
196                        TRASHCANS   


In [55]:
master_remainder.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 15 to 933
Data columns (total 6 columns):
type              184 non-null object
desc              205 non-null object
text              205 non-null object
remedial          15 non-null object
tokenized_desc    205 non-null object
tokenized_text    205 non-null object
dtypes: object(6)
memory usage: 11.2+ KB


In [67]:
remainder_BB = master_remainder[master_remainder.type.str.match(r"BB*", na = False)]
remainder_noBB = master_remainder.drop(master_remainder[master_remainder.type.str.match(r"BB*", na = False)].index)
remainder_noBB

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
15,NaN,N,(B). It is unlawful for any person to dispose ...,1,[N],"[(, B, ), ., It, is, unlawful, for, any, perso..."
438,NaN,,he or she shall serve notice of the risk and t...,NaN,[],"[he, or, she, shall, serve, notice, of, the, r..."
439,NaN,N,tree at the expense of the property owner,2,[N],"[tree, at, the, expense, of, the, property, ow..."
707,NaN,,"human occupation or use, upon failure or refus...",NaN,[],"[human, occupation, or, use, ,, upon, failure,..."
708,NaN,N,"remove or demolish, such dwelling or structure...",2,[N],"[remove, or, demolish, ,, such, dwelling, or, ..."
895,NaN,N,4. A valid identification card showing a re...,10635,[N],"[4, ., A, valid, identification, card, showing..."
897,NaN,,A. Department of Health,NaN,[],"[A, ., Department, of, Health]"
898,NaN,,B. Fire Marshal Approval,NaN,[],"[B, ., Fire, Marshal, Approval]"
899,NaN,,C. Certificate of Registration or Tax Exempt...,NaN,[],"[C., Certificate, of, Registration, or, Tax, E..."


In [70]:
#Some of these might belong in MCL_Chapter (Garbage, Trash, Vermin, Sewage). 
#Check MCL_Chapter if it has to do with something other than houses.
remainder_CA = remainder_noBB[remainder_noBB.type.str.match(r"CA*", na = False)]
remainder_noCA = remainder_noBB.drop(remainder_CA.index)
remainder_noCA

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
15,NaN,N,(B). It is unlawful for any person to dispose ...,1,[N],"[(, B, ), ., It, is, unlawful, for, any, perso..."
438,NaN,,he or she shall serve notice of the risk and t...,NaN,[],"[he, or, she, shall, serve, notice, of, the, r..."
439,NaN,N,tree at the expense of the property owner,2,[N],"[tree, at, the, expense, of, the, property, ow..."
707,NaN,,"human occupation or use, upon failure or refus...",NaN,[],"[human, occupation, or, use, ,, upon, failure,..."
708,NaN,N,"remove or demolish, such dwelling or structure...",2,[N],"[remove, or, demolish, ,, such, dwelling, or, ..."
895,NaN,N,4. A valid identification card showing a re...,10635,[N],"[4, ., A, valid, identification, card, showing..."
897,NaN,,A. Department of Health,NaN,[],"[A, ., Department, of, Health]"
898,NaN,,B. Fire Marshal Approval,NaN,[],"[B, ., Fire, Marshal, Approval]"
899,NaN,,C. Certificate of Registration or Tax Exempt...,NaN,[],"[C., Certificate, of, Registration, or, Tax, E..."


In [75]:
#These might be duplicates for MCL_Chapter or remainder_CA
remainder_HD = remainder_noCA[remainder_noCA.type.str.match(r"HD*", na = False)]
#remainder_HD
remainder_noHD = remainder_noCA.drop(remainder_HD.index)
remainder_noHD

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
15,NaN,N,(B). It is unlawful for any person to dispose ...,1,[N],"[(, B, ), ., It, is, unlawful, for, any, perso..."
438,NaN,,he or she shall serve notice of the risk and t...,NaN,[],"[he, or, she, shall, serve, notice, of, the, r..."
439,NaN,N,tree at the expense of the property owner,2,[N],"[tree, at, the, expense, of, the, property, ow..."
707,NaN,,"human occupation or use, upon failure or refus...",NaN,[],"[human, occupation, or, use, ,, upon, failure,..."
708,NaN,N,"remove or demolish, such dwelling or structure...",2,[N],"[remove, or, demolish, ,, such, dwelling, or, ..."
895,NaN,N,4. A valid identification card showing a re...,10635,[N],"[4, ., A, valid, identification, card, showing..."
897,NaN,,A. Department of Health,NaN,[],"[A, ., Department, of, Health]"
898,NaN,,B. Fire Marshal Approval,NaN,[],"[B, ., Fire, Marshal, Approval]"
899,NaN,,C. Certificate of Registration or Tax Exempt...,NaN,[],"[C., Certificate, of, Registration, or, Tax, E..."


In [78]:
remainder_SW = remainder_noBB[remainder_noBB.type.str.match(r"SW*", na = False)]
#remainder_SW
remainder_noSW = remainder_noHD.drop(remainder_SW.index)
#The remaining entries I think are part of other entries, duplicates, or headings. No real remaining entries left here.
remainder_noSW

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
15,NaN,N,(B). It is unlawful for any person to dispose ...,1,[N],"[(, B, ), ., It, is, unlawful, for, any, perso..."
438,NaN,,he or she shall serve notice of the risk and t...,NaN,[],"[he, or, she, shall, serve, notice, of, the, r..."
439,NaN,N,tree at the expense of the property owner,2,[N],"[tree, at, the, expense, of, the, property, ow..."
707,NaN,,"human occupation or use, upon failure or refus...",NaN,[],"[human, occupation, or, use, ,, upon, failure,..."
708,NaN,N,"remove or demolish, such dwelling or structure...",2,[N],"[remove, or, demolish, ,, such, dwelling, or, ..."
895,NaN,N,4. A valid identification card showing a re...,10635,[N],"[4, ., A, valid, identification, card, showing..."
897,NaN,,A. Department of Health,NaN,[],"[A, ., Department, of, Health]"
898,NaN,,B. Fire Marshal Approval,NaN,[],"[B, ., Fire, Marshal, Approval]"
899,NaN,,C. Certificate of Registration or Tax Exempt...,NaN,[],"[C., Certificate, of, Registration, or, Tax, E..."


In [91]:
master_TCA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 106 to 882
Data columns (total 6 columns):
type              74 non-null object
desc              74 non-null object
text              74 non-null object
remedial          1 non-null object
tokenized_desc    74 non-null object
tokenized_text    74 non-null object
dtypes: object(6)
memory usage: 6.5+ KB


In [84]:
TCA_list = master_TCA["tokenized_desc"].tolist()
TCA_flat_list = [item for sublist in TCA_list for item in sublist]


TCA_kw_freq = dict(Counter(TCA_flat_list))
#TCA_kw_freq

TCA_kw_s = pd.Series(TCA_kw_freq, name="freq")

TCA_kw = pd.DataFrame(data = TCA_kw_s, columns = ["freq"])
TCA_kw = TCA_kw.reset_index()
TCA_kw.columns = ["kw", "freq"]
#TCA_kw.head()
TCA_kw_sorted = TCA_kw.sort_values(by = "freq", ascending = False)
TCA_kw_sorted.head(10)

,kw,freq
62,Supplier,10
182,to,10
8,Beer,10
183,transfer,9
24,Homemade,8
66,Wholesaler,6
151,permit,5
157,require,5
137,not,4
109,for,4


In [92]:
TCA_supplier = master_TCA[master_TCA["desc"].str.contains("supplier", case = False)]
TCA_supplier

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
853,BT5755032A,Supplier coerce wholesaler agreement,"T.C.A 57-5-503(2)(A): Coerce, or attempt to co...",NaN,"[Supplier, coerce, wholesaler, agreement]","[T.C.A, 57-5-503, (, 2, ), (, A, ), :, Coerce,..."
854,BT5755032B,Supplier coerce delivery,"T.C.A 57-5-503(2)(B): Coerce, or attempt to co...",NaN,"[Supplier, coerce, delivery]","[T.C.A, 57-5-503, (, 2, ), (, B, ), :, Coerce,..."
855,BT5755032C,Supplier withhold delivery,T.C.A 57-5-503(2)(C): Withhold delivery of bee...,NaN,"[Supplier, withhold, delivery]","[T.C.A, 57-5-503, (, 2, ), (, C, ), :, Withhol..."
856,BT5755032D,Supplier cannot require brand sales,T.C.A 57-5-503(2)(D): Require a wholesaler to ...,NaN,"[Supplier, can, not, require, brand, sales]","[T.C.A, 57-5-503, (, 2, ), (, D, ), :, Require..."
859,BT5755032G,Supplier require advertising,T.C.A 57-5-503(2)(G): Require a wholesaler by ...,NaN,"[Supplier, require, advertising]","[T.C.A, 57-5-503, (, 2, ), (, G, ), :, Require..."
860,BT5755032H,Supplier require termination,T.C.A 57-5-503(2)(H): Require a wholesaler to ...,NaN,"[Supplier, require, termination]","[T.C.A, 57-5-503, (, 2, ), (, H, ), :, Require..."
861,BT5755032I,Supplier retailatory action,T.C.A 57-5-503(2)(I): Take any retaliatory act...,NaN,"[Supplier, retailatory, action]","[T.C.A, 57-5-503, (, 2, ), (, I, ), :, Take, a..."
862,BT5755032J,Supplier threaten cancel credit,T.C.A 57-5-503(2)(J): Threaten to cancel or wi...,NaN,"[Supplier, threaten, cancel, credit]","[T.C.A, 57-5-503, (, 2, ), (, J, ), :, Threate..."
864,BT575504B,Wholesaler transfer supplier consent,T.C.A 57-5-504(b): A supplier's consent shall ...,NaN,"[Wholesaler, transfer, supplier, consent]","[T.C.A, 57-5-504, (, b, ), :, A, supplier, 's,..."


In [93]:
TCA_beer = master_TCA[master_TCA["desc"].str.contains("beer", case = False)]
TCA_beer

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
831,BT575111B1A,Homemade Beer Compensation,T.C.A 57-5-111(b)(1)(A): No license or permit ...,NaN,"[Homemade, Beer, Compensation]","[T.C.A, 57-5-111, (, b, ), (, 1, ), (, A, ), :..."
832,BT575111B1B,Homemade Beer Sales,T.C.A 57-5-111(b)(1)(B): No license or permit ...,NaN,"[Homemade, Beer, Sales]","[T.C.A, 57-5-111, (, b, ), (, 1, ), (, B, ), :..."
833,BT575111B1C,Homemade Beer exceed gallon limit,T.C.A 57-5-111(b)(1)(C): No license or permit ...,NaN,"[Homemade, Beer, exceed, gallon, limit]","[T.C.A, 57-5-111, (, b, ), (, 1, ), (, C, ), :..."
834,BT575111B3,Homemade Beer not identified as homemade,T.C.A 57-5-111(b)(3): Any homemade beer in com...,NaN,"[Homemade, Beer, not, identified, as, homemade]","[T.C.A, 57-5-111, (, b, ), (, 3, ), :, Any, ho..."
835,BT575111C,Homemade Beer Exhibition,T.C.A 57-5-111(c): The use of homemade beer ma...,NaN,"[Homemade, Beer, Exhibition]","[T.C.A, 57-5-111, (, c, ), :, The, use, of, ho..."
836,BT575111D1,Homemade Beer Contests,T.C.A 57-5-111(d)(1): Notwithstanding any law ...,NaN,"[Homemade, Beer, Contests]","[T.C.A, 57-5-111, (, d, ), (, 1, ), :, Notwith..."
837,BT575111D2,Homemade Beer Contest Fees,T.C.A 57-5-111(d)(2): No fee may be charged fo...,NaN,"[Homemade, Beer, Contest, Fees]","[T.C.A, 57-5-111, (, d, ), (, 2, ), :, No, fee..."
838,BT575111D3,Homemade Beer Storage on permitted site,"T.C.A 57-5-111(d)(3): If the contest, competit...",NaN,"[Homemade, Beer, Storage, on, permitted, site]","[T.C.A, 57-5-111, (, d, ), (, 3, ), :, If, the..."
839,BT575112A,Flavored Beer labeling,T.C.A 57-5-112(a): Any wholesaler or retailer ...,NaN,"[Flavored, Beer, labeling]","[T.C.A, 57-5-112, (, a, ), :, Any, wholesaler,..."


In [97]:
TCA_noSuppBeer = master_TCA.drop((TCA_supplier | TCA_beer).index)
TCA_noSuppBeer

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
106,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG,"[Landlord, Registration, Required]","[Tennessee, Code, Annotated, (, TCA, ), 66-28-..."
376,CAAR_LAND_REG,LANDLORD REGISTRATION REQUIRED,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,NaN,"[LANDLORD, REGISTRATION, REQUIRED]","[Tennessee, Code, Annotated, (, TCA, ), 66-28-..."
807,BT575101A2,Not Manufacturer and Wholesale Retail,T.C.A 57-5-101(a)(2): Except as otherwise prov...,NaN,"[Not, Manufacturer, and, Wholesale, Retail]","[T.C.A, 57-5-101, (, a, ), (, 2, ), :, Except,..."
808,BT575101A3,Not Wholesale and Manufacturer Retail,T.C.A 57-5-101(a)(3): Except as otherwise prov...,NaN,"[Not, Wholesale, and, Manufacturer, Retail]","[T.C.A, 57-5-101, (, a, ), (, 3, ), :, Except,..."
809,BT575103A1,Operating without county city permit,T.C.A 57-5-103(a)(1): It is unlawful to operat...,NaN,"[Operating, without, county, city, permit]","[T.C.A, 57-5-103, (, a, ), (, 1, ), :, It, is,..."
810,BT575103A10,Permits to citizen legal resident,"T.C.A 57-5-103(a)(10): After July 1, 2015, a c...",NaN,"[Permits, to, citizen, legal, resident]","[T.C.A, 57-5-103, (, a, ), (, 10, ), :, After,..."
811,BT575103A2,Permit to business owner,T.C.A 57-5-103(a)(2): Permits shall be issued ...,NaN,"[Permit, to, business, owner]","[T.C.A, 57-5-103, (, a, ), (, 2, ), :, Permits..."
812,BT575103A3A,Cannot transfer ownership,T.C.A 57-5-103(a)(3)(A): A permit shall be val...,NaN,"[Can, not, transfer, ownership]","[T.C.A, 57-5-103, (, a, ), (, 3, ), (, A, ), :..."
813,BT575103A3B,Permit for single location only,T.C.A 57-5-103(a)(3)(B): A permit shall be val...,NaN,"[Permit, for, single, location, only]","[T.C.A, 57-5-103, (, a, ), (, 3, ), (, B, ), :..."


In [98]:
TCA_noSuppBeer[TCA_noSuppBeer["desc"].str.contains("wholesale", case = False)]

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
807,BT575101A2,Not Manufacturer and Wholesale Retail,T.C.A 57-5-101(a)(2): Except as otherwise prov...,NaN,"[Not, Manufacturer, and, Wholesale, Retail]","[T.C.A, 57-5-101, (, a, ), (, 2, ), :, Except,..."
808,BT575101A3,Not Wholesale and Manufacturer Retail,T.C.A 57-5-101(a)(3): Except as otherwise prov...,NaN,"[Not, Wholesale, and, Manufacturer, Retail]","[T.C.A, 57-5-101, (, a, ), (, 3, ), :, Except,..."
851,BT5755031A,Wholesaler transfer written notice,T.C.A 57-5-503(1)(A): Transfer the wholesaler'...,NaN,"[Wholesaler, transfer, written, notice]","[T.C.A, 57-5-503, (, 1, ), (, A, ), :, Transfe..."
852,BT5755031B,Wholesale transfer without approval,T.C.A 57-5-503(1)(B): Transfer the wholesaler'...,NaN,"[Wholesale, transfer, without, approval]","[T.C.A, 57-5-503, (, 1, ), (, B, ), :, Transfe..."
863,BT575504A,Wholesaler written notice,T.C.A 57-5-504(a): A wholesaler shall give the...,NaN,"[Wholesaler, written, notice]","[T.C.A, 57-5-504, (, a, ), :, A, wholesaler, s..."
866,BT575504C1,Wholesaler transfer to felony conviction,T.C.A 57-5-504(c)(1): The transferee or any ow...,NaN,"[Wholesaler, transfer, to, felony, conviction]","[T.C.A, 57-5-504, (, c, ), (, 1, ), :, The, tr..."
867,BT575504C2,Wholesale transfer to revoked suspended,T.C.A 57-5-504(c)(2): The transferee or any ow...,NaN,"[Wholesale, transfer, to, revoked, suspended]","[T.C.A, 57-5-504, (, c, ), (, 2, ), :, The, tr..."
868,BT575504C3,Wholesaler transfer bankruptcy,T.C.A 57-5-504(c)(3): The transferee or any ow...,NaN,"[Wholesaler, transfer, bankruptcy]","[T.C.A, 57-5-504, (, c, ), (, 3, ), :, The, tr..."
869,BT575504C4,Wholesaler previous agreement cancelled,T.C.A 57-5-504(c)(4): The transferee or any ow...,NaN,"[Wholesaler, previous, agreement, cancelled]","[T.C.A, 57-5-504, (, c, ), (, 4, ), :, The, tr..."
